In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import time
from datetime import datetime
from functools import reduce
import calendar
from haversine import haversine, Unit
from scipy.optimize import linear_sum_assignment
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Load the csv with hyderabad area1 hexes
hyderabad_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone1.csv')
hyderabad_zone1_hexes = list(hyderabad_zone1['customer_location_hex_8'].unique())

In [5]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Hyderabad')
and resolution = 8
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [6]:
#Create a copy of the df
df = df_copy.copy()
df.head()

,city,cluster,hex_id,cluster_original,resolution,executiondate
0,Hyderabad,Banjara Hills,8860a25919fffff,Banjara Hills,8,2022-11-01
1,Hyderabad,Shamirpet,8860b19097fffff,Shamirpet,8,2022-11-01
2,Hyderabad,AOC Center,8860b52d0bfffff,AOC Center,8,2022-11-01
3,Hyderabad,AOC Center,8860b52d9bfffff,AOC Center,8,2022-11-01
4,Hyderabad,Tellapur,8860a248c9fffff,Tellapur,8,2022-11-01


In [8]:
#Identify hexes in area1
df['zone_name'] = np.where(df['hex_id'].isin(hyderabad_zone1_hexes), 'zone1', 'rest_of_hyderabad')
df_zone1 = df[df['zone_name'] == 'zone1']
df_cluster_grouped = df_zone1.groupby(['cluster'])['hex_id'].count().reset_index()
df_cluster_grouped2 = df_zone1.groupby(['cluster'])['hex_id'].apply(list).reset_index()
df_cluster_grouped = df_cluster_grouped.rename(columns = {'hex_id': 'count_of_hex_ids'})
df_cluster_grouped = df_cluster_grouped.merge(df_cluster_grouped2, on = 'cluster', how ='left')
df_cluster_grouped

,cluster,count_of_hex_ids,hex_id
0,Gachibowli,8,"[8860a24a6bfffff, 8860a25991fffff, 8860a24a63f..."
1,Hitech City,9,"[8860a259b3fffff, 8860a24a61fffff, 8860a259b1f..."
2,Jubilee Hills,1,[8860a25983fffff]
3,Kondapur,4,"[8860a24a67fffff, 8860a24b53fffff, 8860a24a29f..."
4,Kothaguda,6,"[8860a24b57fffff, 8860a24b55fffff, 8860a24b51f..."
5,Madhapur,3,"[8860a259bdfffff, 8860a259abfffff, 8860a25987f..."
6,Mothinagar,1,[8860a259a3fffff]


In [9]:
df_cluster_grouped.to_csv('cluster_names_hyderabad_zone1.csv', index = False)